# Administrador en Django

Existen varias formas de manejar los registros de la base de datos, en esta introducción nos limitaremos a utilizar el panel de administrador y experimentar un poco con la `shell`, un entorno que permite ejecutar instrucciones directamente en el proyecto.

Para acceder al administrador tenemos que acceder a la URL `/admin`. Esta dirección no es casual, está definida así dentro del fichero `urls.py` de nuestro proyecto: http://127.0.0.1:8000/admin/login/?next=/admin/

Nos pedirá identificarnos un usuario y una contraseña. Este usuario no puede ser cualquiera, tiene que tener permisos. Podemos crear un primer *superuser* con todos los permisos desde la terminal:

```bash
$ pipenv run python manage.py createsuperuser
```

Una vez creado si nos identificarnos accederemos al panel de administración.

Este panel contiene diferentes bloques y cada uno corresponde a los modelos de un app de Django.

Solo tenemos un bloque activo llamado **Autenticación y autorización**, es el de la app `auth`, encargada de manejar los grupos de permisos y usuarios.

Como véis sin hacer nada ya contamos con un panel donde podemos manejar usuarios cómodamente y añadirles permisos para manejar otras apps. Sin embargo nuestra app `blog` todavía no aparece, eso es porque tenemos que activar los modelos que queramos manejar en el administrador.

Vamos a activar el admin para el modelo `Post`, podemos hacerlo desde el fichero `blog/admin.py`:

`blog/admin.py`

```python
from django.contrib import admin
from .models import Post

admin.site.register(Post)
```

Con este cambio os aparecerá el bloque de la app `blog` y ya podemos empezar a crear, editar y borrar `Post` utilizando dos formularios: el de modelo y el de listado.

## Formulario de modelo

Podemos personalizar los campos que se muestran en el formulario del modelo, por ejemplo el atributo `verbose_name` permite cambiar el nombre del campo, podemos usarlo para traducirlo:

`blog/models.py`

```python
class Post(models.Model):
    title = models.CharField(max_length=200, verbose_name="Título")
    content = models.TextField(verbose_name="Contenido")
    created = models.DateTimeField(auto_now_add=True, verbose_name="Creado")
    modified = models.DateTimeField(auto_now=True, verbose_name="Modificado")
```

También podemos cambiar el nombre del modelo global, tanto en singular como plural:

```python
class Post(models.Model):

    # nuevo
    class Meta:
        verbose_name = "entrada"
        verbose_name_plural = "entradas"
```

Por cierto, ¿os habéis fijado que las fechas de creación y modificación no se muestran? Por defecto los campos `auto_now_add` y `auto_now` son automáticos y se esconden, podemos configurar que aparezca creando una configuración extendida indicándolos como campos solo de lectura:

`blog/admin.py`

```python
class PostAdmin(admin.ModelAdmin):
    readonly_fields = ('created', 'modified')

admin.site.register(Post, PostAdmin)
```

## Formulario de listado

Por defecto en la lista de entradas se hace un *cast* del objeto a cadena haciendo que aparezca una lista muy fea y poco intuitiva. Si sobreescribimos el método *string* del modelo y devolvemos el título quedará mejor:

`blog/models.py`

```python
class Post(models.Model):
    # nuevo
    def __str__(self):
        return self.title
```

También podemos cambiar al modo de visualización por columnas:

`blog/admin.py`

```python
class PostAdmin(admin.ModelAdmin):
    readonly_fields = ('created', 'modified')
    list_display = ('title', 'created', 'modified')  # nuevo
```

O aprovechando que tenemos campos de fecha, podemos establecer una jerarquía por fechas:

```python
class PostAdmin(admin.ModelAdmin):
    readonly_fields = ('created', 'modified')
    list_display = ('title', 'created', 'modified')
    date_hierarchy = 'created'  # nuevo
```

Hay mil cosas que podemos configurar, si queréis saber más consultad la [documentación oficial](https://docs.djangoproject.com/en/dev/ref/contrib/admin/).